## Relayed HOPR tokens by the ct-app

In [2]:
from dotenv import load_dotenv
import sys
sys.path.append("../")

from core.components.parameters import Parameters # noqa: E402
from core.components.utils import Utils # noqa: E402
from core.model.subgraph_entry import SubgraphEntry # noqa: E402
from core.model.topology_entry import TopologyEntry # noqa: E402
from core.model.economic_model import EconomicModel # noqa: E402
from core.model.peer import Peer # noqa: E402
from core.node import Node # noqa: E402

In [2]:
# Load environment variables
load_dotenv(".envs/relayed_token.env")

params = Parameters()("SUBGRAPH_", "GCP_", "ECONOMIC_MODEL_")
nodes = Node.fromAddressListAndKey(*Utils.nodesAddresses("NODE_ADDRESS_", "NODE_KEY"))

api = nodes[-1].api
network_nodes = nodes[:-1]


In [3]:
async def get_subgraph_data():
    data = {
        "query": params.subgraph.safes_balance_query,
        "variables": {"first": params.subgraph.pagination_size, "skip": 0},
    }

    safes = []
    while True:
        _, response = await Utils.httpPOST(
            params.subgraph.safes_balance_url, data
        )

        if "data" not in response:
            break

        safes.extend(response["data"]["safes"])

        if len(response["data"]["safes"]) >= params.subgraph.pagination_size:
            data["variables"]["skip"] += params.subgraph.pagination_size
        else:
            break

    results = list[SubgraphEntry]()
    for safe in safes:
        results.extend(
            [
                SubgraphEntry.fromSubgraphResult(node)
                for node in safe["registeredNodesInNetworkRegistry"]
            ]
        )


    return results

async def get_topology_data():
    channels = await api.all_channels(False)

    results = await Utils.aggregatePeerBalanceInChannels(channels.all)
    return [TopologyEntry.fromDict(*arg) for arg in results.items()]

async def get_node_data():
    results = set[Peer]()

    for node in network_nodes:
        await node._retrieve_address()
        node_result = await node.api.peers(params=["peer_id", "peer_address"], quality=0.5)
        
        peers = {Peer(item["peer_id"], item["peer_address"]) for item in node_result}
        results.update(peers)

    return results

async def get_fundings(from_address: str, to_address: str):
    query: str =  params.subgraph.wxhopr_txs_query
    query = query.replace("$from", f'\"{from_address}\"')
    query = query.replace("$to", f'\"{to_address}\"')

    _, response = await Utils.httpPOST(params.subgraph.wxhopr_txs_url, { "query": query })

    amounts = [float(tx["amount"]) for tx in response["data"]["transactions"]] 

    return amounts

In [4]:
topology = await get_topology_data()
print(f"Topology size: {len(topology)}")

subgraph = await get_subgraph_data()
print(f"Subgraph size: {len(subgraph)}")

peers = await get_node_data()
print(f"Number of peers: {len(peers)}")

Topology size: 397
Subgraph size: 552
Number of peers: 352


In [5]:
eligible = Utils.mergeTopologyPeersSubgraph(topology, peers, subgraph)
Utils.allowManyNodePerSafe(eligible)
model = EconomicModel.fromGCPFile(
    params.gcp.bucket, params.economic_model.filename
)
for peer in eligible:
    peer.economic_model = model
    
Utils.rewardProbability(eligible)

print(f"Eligible peers: {len(eligible)}")

Eligible peers: 287


### Funds of Netwatchers 

In [6]:
node_funds = []

for node in network_nodes:
    peer = [peer for peer in eligible if peer.address == node.address][0]
    node_funds.append(peer.total_balance)

print(node_funds)

[72600.9, 72699.6, 73160.9, 73813.5]


### Relayed Token Calculation 

In [11]:
ct_nodes = []
for node in network_nodes:
    node = [e for e in eligible if e.address == node.address][0]
    ct_nodes.append(node)

ct_funding_received=0
for node in ct_nodes:
    amounts = await get_fundings("0xd9a00176cf49dfb9ca3ef61805a2850f45cb1d05", node.safe_address)
    ct_funding_received += sum(amounts)

distributed_rewards = ct_funding_received - sum(node_funds)
print(f"{ct_funding_received=:_.2f}")
print(f"{distributed_rewards=:_.2f}")

ct_funding_received=350_000.00
distributed_rewards=57_725.10


In [13]:
airdropped = 6*25_000 # Get this number from Andrius
total_rewards = airdropped + distributed_rewards
print(f"{total_rewards=:_.2f}")

total_rewards=207_725.10
